# Using an output handler for validating agent's output

As no one can guarantee the validity or integrity of a raw LLM output, getting a sensible and/or structured output from an agent is tricky. In motleycrew, we addressed this issue with a novel approach: please welcome **output handlers**.

As of now, tool calling is the most reliable way to get a structured output from an LLM. So why not have an output handler - a regular tool with an input schema and a description which instructs the agent to return the output only via this tool. Moreover, we can force the agent into doing this: if it attempts to finish the regular way, we intercept the message and remind the agent that it must call the tool instead.

Let's install external dependencies for the example.

In [1]:
%pip install ddgs

# install lunary if you need logging
%pip install lunary==1.1.5


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached lunary-1.1.5-py3-none-any.whl.metadata (1.6 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached setuptools-72.2.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached tenacity-8.5.0-py3-none-any.whl.metadata (1.2 kB)
Using cached lunary-1.1.5-py3-none-any.whl (18 kB)
Using cached packaging-23.2-py3-none-any.whl (53 kB)
Using cached setuptools-72.2.0-py3-none-any.whl (2.3 MB)
Using cached tenacity-8.5.0-py3-none-any.whl (28 kB)
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.1.2
    Uninstalling tenacity-9.1.2:
      Successfully uninstalled tenacity-9.1.2
  Attempting uninstall: setuptools
    Found existing installation: setuptools 80.9.0
    Uninstalling setuptools-80.9.0:
      Successfully uninstalled setuptools-80.9.0
  Attempting un

In [2]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.tools import StructuredTool

from dotenv import load_dotenv
load_dotenv()

from motleycrew import MotleyCrew
from motleycrew.agents.langchain import ReActToolCallingMotleyAgent
from motleycrew.common import configure_logging
from motleycrew.tasks import SimpleTask

from motleycrew.common.exceptions import InvalidOutput

from motleycrew.tools import MotleyTool, RetryConfig

We'll be prompting our agent to create a report on the latest advancements in AI.  
Say we want it to also include some info on AI applications in physics if it did not already. So our output handler will look for the keyword "physics" in the agent's output.

Also, we can do any data manipulations we want inside the output handler. In this case, let's just place the output into a certain field.

In [3]:
def check_output(output: str):
    if "physics" not in output.lower():
        raise InvalidOutput("Add more information about AI applications in physics.")

    return {"checked_output": output}

output_handler = StructuredTool.from_function(
    name="output_handler",
    description="Output handler",
    func=check_output,
)  # Schema is inferred from the function signature, but you can also specify it explicitly

# To set return_direct=True, we'll convert it into a MotleyTool
output_handler = MotleyTool.from_supported_tool(output_handler, return_direct=True)

Now let's create our agent and launch the crew!

In [4]:
search_tool = MotleyTool.from_supported_tool(
    DuckDuckGoSearchRun(),
    retry_config=RetryConfig(max_retries=7)  # for retrying rate limit errors
)

researcher = ReActToolCallingMotleyAgent(
    tools=[search_tool, output_handler],
    verbose=True,
    chat_history=True,
    force_output_handler=True,  # This will force the agent to use the output handler instead of regular output
)

crew = MotleyCrew()
task = SimpleTask(
    crew=crew,
    name="produce comprehensive analysis report on AI advancements in 2024",
    description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
Identify key trends, breakthrough technologies, and potential industry impacts.
Perform no more than 3 search queries. Your final answer MUST be a full analysis report""",
    agent=researcher,
)

In [5]:
crew.run()



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'key AI trends 2024'}`
responded: Thought: To conduct a comprehensive analysis of the latest advancements in AI in 2024, I need to gather information on key trends, breakthrough technologies, and potential industry impacts. I will perform three search queries to gather this information. The first query will focus on identifying the key trends in AI for 2024. The second query will look for breakthrough technologies in AI. The third query will explore the potential industry impacts of these advancements. 

I will start with the first search query to identify key trends in AI for 2024. 

Let's proceed with the first search query.



/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Dec 11, 2024 · In light of this, AI Magazine presents a comprehensive list of the top 10 AI trends of 2024 to understand the AI developments of this year and gain a glimpse of what could be in … Jan 29, 2025 · Discover the latest AI adoption stats and ROI insights for 2024. Explore how AI is transforming industries, boosting productivity, and shaping the future. Aug 6, 2024 · As we navigate through 2024, several key AI trends are emerging, each with significant implications for businesses and society. Here, we delve into 15 prominent AI trends … Sep 30, 2024 · Explore AI-driven personalization, autonomous systems, NLP, and ethical challenges shaping AI in 2024. Nov 12, 2024 · Below is a look at the top trends we’re seeing in IDC’s study and the impact of those trends on organizations working with AI today. #1 Enhanced productivity has become …
Invoking: `duckduckgo_search` with `{'query': 'breakthrough AI technologies 2024'}`
responded: Thought: I have gathered some initial information

/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Dec 26, 2024 · Dive into 2024's most significant AI breakthroughs and controversies, from Apple's privacy-focused intelligence to Google's quantum leap. Explore how autonomous agents … Dec 20, 2024 · Explore the top AI news of 2024, from copyright lawsuits to generative AI product launches, and discover what will shape the industry's future in 2025. Apr 1, 2025 · 2024 was a landmark year for artificial intelligence, with several significant advancements that have the potential to transform various industries. From Meta's AI-powered … Oct 7, 2024 · With its exponential growth, AI is now integral to daily life—from powering social media to enhancing e-commerce and education. Below, we explore the top 10 AI technologies … Nov 22, 2024 · In 2024, technology is being revolutionised by ten key breakthroughs including AI advancements, super-efficient solar cells, Apple's Vision Pro, innovative weight-loss drugs...
Invoking: `duckduckgo_search` with `{'query': 'AI industry impacts 2024'}`
resp

...

目前的 AI大模型本质是，以统计规律代替逻辑规律，以相关性代替因果性，以海量参数的函数拟合出输入输出算法。 具体的技术原理：1 是通过数据集获得统计规律，2是通过统计规律进行插值输出。 1）Cursor Cursor是国外一家公司在2023年发布的一款AI原生的IDE，主要功能就是通过AI能力自动检索理解代码的上下文、可以自动编写并运行终端命令、可以自动检测并修正代码、具有强大的自动补全 … ①Sora： AI视频生成模型，可生成长达一分钟的高质量视频，还未正式对外开放 ②Runway： runwayml.com 强大的AI视频制作工具，绿幕抠像、视频合成等 ③D-ID： d-id.com AI真人口播视频生 … Jan 31, 2025 · 目前出现这种情况，是因为 deepseek 受到了 ddos 网络攻击，目前作为普通用户，只能等待危机解除以后再来使用了。 期待 deepseek 能扛过攻击，早日强大起来，给人们带来更加让人 … 2025年运行AI大模型显卡选择AMD还是英伟达呢？ AMD经过这2年的更新AI性能有没有追上英伟达呢？ 想配一台能跑AI的电脑，来咨询一下，毕竟都说AMD性价比比较高，显存大。 显示全部 关注者 54  …
Invoking: `duckduckgo_search` with `{'query': 'AI advancements in physics 2024'}`
responded: Thought: The search did not yield specific information about AI applications in physics for 2024. I will refine the search query to focus more specifically on AI advancements in the field of physics, which may include areas like quantum physics, particle physics, or computational physics.

Let's proceed with a refined search query to gather this information.



/Users/whimo/motleycrew/.venv/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


目前的 AI大模型本质是，以统计规律代替逻辑规律，以相关性代替因果性，以海量参数的函数拟合出输入输出算法。 具体的技术原理：1 是通过数据集获得统计规律，2是通过统计规律进行插 … 1）Cursor Cursor是国外一家公司在2023年发布的一款AI原生的IDE，主要功能就是通过AI能力自动检索理解代码的上下文、可以自动编写并运行终端命令、可以自动检测并修正代码、具有强 … ①Sora： AI视频生成模型，可生成长达一分钟的高质量视频，还未正式对外开放 ②Runway： runwayml.com 强大的AI视频制作工具，绿幕抠像、视频合成等 ③D-ID： d-id.com AI真人口播 … Jan 31, 2025 · 目前出现这种情况，是因为 deepseek 受到了 ddos 网络攻击，目前作为普通用户，只能等待危机解除以后再来使用了。 期待 deepseek 能扛过攻击，早日强大起来，给人们带 … 2025年运行AI大模型显卡选择AMD还是英伟达呢？ AMD经过这2年的更新AI性能有没有追上英伟达呢？ 想配一台能跑AI的电脑，来咨询一下，毕竟都说AMD性价比比较高，显存大。 显示全部  …


Invoking: `output_handler` with `{'output': "### Comprehensive Analysis of AI Advancements in 2024\n\n#### Key AI Trends in 2024\n1. **AI-Driven Personalization**: AI is increasingly being used to tailor experiences and products to individual preferences, enhancing user engagement and satisfaction.\n2. **Autonomous Systems**: The development of autonomous vehicles and drones continues to advance, with significant improvements in safety and efficiency.\n3. **Natural Language Processing (NLP)**: NLP technologies are becoming more sophisticated, enabling more natural and intuitive interactions between humans and machines.\n4. **Ethical AI**: There is a growing focus on addressing ethical challenges in AI, including bias, privacy, and transparency.\n5. **Enhanced Productivity**: AI is being leveraged to boost productivity across various sectors, from manufacturing to healthcare.\n\n#### Breakthrough AI Technologies in 2024\n1. **Privacy-Focused Intelligence**: Companies like Apple are lea

[TaskUnit(status=done)]

As you can see in the logs, our physics requirement was successfully fed back to the agent!

In [6]:
from IPython.display import display, Markdown

final_result = task.output["checked_output"]
display(Markdown(final_result))

### Comprehensive Analysis of AI Advancements in 2024

#### Key AI Trends in 2024
1. **AI-Driven Personalization**: AI is increasingly being used to tailor experiences and products to individual preferences, enhancing user engagement and satisfaction.
2. **Autonomous Systems**: The development of autonomous vehicles and drones continues to advance, with significant improvements in safety and efficiency.
3. **Natural Language Processing (NLP)**: NLP technologies are becoming more sophisticated, enabling more natural and intuitive interactions between humans and machines.
4. **Ethical AI**: There is a growing focus on addressing ethical challenges in AI, including bias, privacy, and transparency.
5. **Enhanced Productivity**: AI is being leveraged to boost productivity across various sectors, from manufacturing to healthcare.

#### Breakthrough AI Technologies in 2024
1. **Privacy-Focused Intelligence**: Companies like Apple are leading the way in developing AI technologies that prioritize user privacy.
2. **Quantum Computing**: Google's advancements in quantum computing are paving the way for more powerful AI applications.
3. **Generative AI**: New generative AI models are being launched, capable of creating high-quality content and media.
4. **AI-Powered Social Media**: AI is being integrated into social media platforms to enhance user experience and content delivery.
5. **AI in E-commerce and Education**: AI technologies are revolutionizing e-commerce and education by providing personalized recommendations and learning experiences.

#### Potential Industry Impacts
1. **Healthcare**: AI is transforming healthcare by improving diagnostics, personalizing treatment plans, and streamlining administrative processes.
2. **Automotive**: The automotive industry is seeing significant changes with the rise of autonomous vehicles, which promise to improve safety and reduce traffic congestion.
3. **Retail**: AI is enhancing the retail experience through personalized shopping experiences and efficient inventory management.
4. **Finance**: In the finance sector, AI is being used for fraud detection, risk management, and personalized financial advice.
5. **Manufacturing**: AI-driven automation is increasing efficiency and reducing costs in manufacturing processes.

#### AI Applications in Physics
While specific advancements in AI applications in physics for 2024 were not detailed in the available data, AI continues to play a crucial role in areas such as quantum physics, particle physics, and computational physics. AI is used to analyze complex datasets, simulate physical phenomena, and optimize experimental processes.

In conclusion, 2024 has been a pivotal year for AI, with significant advancements in technology and widespread impacts across various industries. The focus on ethical AI and privacy, along with breakthroughs in quantum computing and generative AI, are shaping the future of AI and its applications.